In [1]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("../")
from src.utils import *
from src.graph2vec import ExtendedGraph2Vec, Ensemble_G2V
from sklearn.model_selection import LeaveOneOut
import pandas as pd

In [2]:
def which_to_generate(num:int):
    path="../datasets/github_stargazers" if num==0 else "../datasets/regular_graps/" if num==1 else "../datasets/partition_graphs/"
    graphs, labels = read_stargazers_dataset("../datasets/github_stargazers") if num==0 else load_artificial(path)
    order_dict = np.load(f"{path}orderings.npy", allow_pickle=True)[()]
    return graphs,labels,order_dict,path

In [3]:
DATASET_NUMBER=2

In [4]:
# cv_fold = 64

graphs, labels,order_dict,path_to_save=which_to_generate(DATASET_NUMBER)

cv_fold,single_emb_size=64,124 if DATASET_NUMBER==0 else 105,2 if DATASET_NUMBER==1 else 100,20

tested_range = np.linspace(0, 1, 11)

WEIGHTING_FUNCTIONS = (
    {f"w_mean_{a:.2f}": ("w_mean", [a]) for a in tested_range}
    | {f"concat": ("concatenate", [])}
    | {f"p_proj_{a}": ("partial_projection", [0, a]) for a in (True, False)}
    | {f"w_proj_{a:.2f}": ("w_projection", [a]) for a in tested_range}
)

ens_model = Ensemble_G2V(ExtendedGraph2Vec(dimensions=single_emb_size),ExtendedGraph2Vec(use_pv_dm=True,dimensions=single_emb_size))

  0%|          | 0/900 [00:00<?, ?it/s]

ValueError: too many values to unpack (expected 2)

In [ ]:
res = []
for measure, order in tqdm(order_dict.items()):
    for name, (func, args) in tqdm(WEIGHTING_FUNCTIONS.items(), leave=False):
        ens_model.set_weighting_function(func, *args)
        res.append(
            cross_validate_graphs(
                graphs=graphs,
                ordering=order,
                labels=labels,
                n_splits=cv_fold,
                embedder=ens_model,
                cls=LogisticRegression(max_iter=1000),
                method=f"{measure}_{name}",
            )
        )

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
pd.concat(res, ignore_index=True).to_csv(f"{path_to_save}results2.csv")
pd.read_csv(f"{path_to_save}results2.csv").sample(15)

,Unnamed: 0,graph_id,method,binary_prediction,true_label,probability_prediction_0,probability_prediction_1,probability_prediction_2
38770,38770,70,closeness_centrality_w_proj_0.40,0,0,0.589159,0.121353,0.289488
64901,64901,101,current_flow_closeness_centrality_w_proj_0.80,2,0,0.420875,0.111172,0.467953
58220,58220,620,current_flow_closeness_centrality_w_proj_0.00,2,2,0.202203,0.100080,0.697717
52800,52800,600,current_flow_closeness_centrality_w_mean_0.80,2,2,0.306494,0.108633,0.584872
48906,48906,306,current_flow_closeness_centrality_w_mean_0.40,2,2,0.376102,0.113332,0.510566
49445,49445,845,current_flow_closeness_centrality_w_mean_0.40,0,0,0.475508,0.127786,0.396706
47681,47681,881,current_flow_closeness_centrality_w_mean_0.20,2,2,0.398508,0.114377,0.487115
4879,4879,379,pagerank_w_mean_0.50,0,0,0.590183,0.114328,0.295489
64594,64594,694,current_flow_closeness_centrality_w_proj_0.70,2,2,0.280257,0.091438,0.628305
18494,18494,494,pagerank_w_proj_0.60,2,1,0.380208,0.122780,0.497012
